# load the zip file
- we load the zip file
- then remove data we don't need
- do some translation

In [1]:
import pandas as pd
df = pd.DataFrame()
df = pd.read_csv("../data/rdw.zip", 
           compression="zip")

In [2]:
df.rename(columns={'kenteken': 'plate'}, inplace=True)
df.rename(columns={'merk': 'make'}, inplace=True)
df.rename(columns={'handelsbenaming': 'model'}, inplace=True)
df.rename(columns={'lengte': 'length'}, inplace=True)
df.rename(columns={'breedte': 'width'}, inplace=True)
df.rename(columns={'inrichting': 'type'}, inplace=True)
df.rename(columns={'voertuigsoort': 'kind'}, inplace=True)
df.rename(columns={'bruto_bpm': 'max_load'}, inplace=True)
df.rename(columns={'europese_voertuigcategorie': 'eu_class'}, inplace=True)

omit = ['vervaldatum_apk','datum_tenaamstelling','aantal_zitplaatsen','max_load','eerste_kleu','tweede_kleur',
        'aantal_cilinders','cilinderinhoud','aanhangwagen_autonoom_geremd','aanhangwagen_middenas_geremd','vervaldatum_tachograaf',
        'vervaldatum_tachograaf_dt','type_gasinstallatie','oplegger_geremd','afwijkende_maximum_snelheid','verticale_belasting_koppelpunt_getrokken_voertuig',
        'aantal_staanplaatsen','subcategorie_nederland','eerste_kleur','massa_ledig_voertuig','toegestane_maximum_massa_voertuig','massa_rijklaar',
        'maximum_massa_trekken_ongeremd','api_gekentekende_voertuigen_carrosserie_specifiek','api_gekentekende_voertuigen_voertuigklasse',
        'europese_voertuigcategorie_toevoeging','europese_uitvoeringcategorie_toevoeging','registratie_datum_goedkeuring_afschrijvingsmoment_bpm',
        'registratie_datum_goedkeuring_afschrijvingsmoment_bpm_dt','laadvermogen','maximum_last_onder_de_vooras_sen_tezamen_koppeling','maximale_constructiesnelheid',
        'hoogte_voertuig','maximum_trekken_massa_geremd','datum_eerste_toelating','datum_eerste_tenaamstelling_in_nederland','wacht_op_keuren',
        'catalogusprijs','code_toelichting_tellerstandoordeel','tenaamstellen_mogelijk','vervaldatum_apk_dt','datum_tenaamstelling_dt',
        'datum_eerste_toelating_dt','datum_eerste_tenaamstelling_in_nederland_dt','zuinigheidsclassificatie','api_gekentekende_voertuigen_assen',
        'api_gekentekende_voertuigen_brandstof','api_gekentekende_voertuigen_carrosserie','wam_verzekerd','aantal_deuren','aantal_wielen',
        'afstand_hart_koppeling_tot_achterzijde_voertuig','afstand_voorzijde_voertuig_tot_hart_koppeling','vermogen_massarijklaar','wielbasis',
        'export_indicator','openstaande_terugroepactie_indicator','taxi_indicator','maximum_massa_samenstelling','aantal_rolstoelplaatsen',
        'maximum_ondersteunende_snelheid','jaar_laatste_registratie_tellerstand','tellerstandoordeel','plaats_chassisnummer','technische_max_massa_voertuig','type',
        'typegoedkeuringsnummer','variant','uitvoering','volgnummer_wijziging_eu_typegoedkeuring','hoogte_voertuig_minimum','hoogte_voertuig_maximum',
        'wielbasis_voertuig_minimum','wielbasis_voertuig_maximum','lengte_voertuig_minimum','lengte_voertuig_maximum','breedte_voertuig_minimum','breedte_voertuig_maximum',
       'breedte_voertuig_maximum','massa_bedrijfsklaar_minimaal','massa_bedrijfsklaar_maximaal']

for item in omit:
    try:
        df.drop(item, axis=1, inplace=True)
    except:
        pass
df.head(100)

,Unnamed: 0,plate,kind,make,model,length,width,eu_class
0,0,N856XD,Personenauto,FIAT,FIAT 500,357.0,163.0,M1
1,1,36DVLX,Personenauto,BMW,3ER REIHE,0.0,0.0,M1
2,2,34XSGJ,Personenauto,HONDA,CIVIC 5DR,425.0,0.0,M1
3,3,HXJV75,Personenauto,JAGUAR,SOVEREIGN 3.2 E2,0.0,0.0,M1
4,4,45TTJZ,Personenauto,CITROEN,CITROEN C1,344.0,0.0,M1
...,...,...,...,...,...,...,...,...
95,95,93JJLN,Personenauto,VOLKSWAGEN,PASSAT,0.0,0.0,M1
96,96,12YD61,Motorfiets,KREIDLER,K54/54,NaN,NaN,L3
97,97,HN344R,Personenauto,VOLKSWAGEN,POLO,NaN,NaN,M1
98,98,JJ012S,Personenauto,VOLKSWAGEN,GOLF,0.0,0.0,M1


Now let's investigate the dataset a little ...

In [3]:
df.describe()

,Unnamed: 0,length,width
count,5000.000000,4487.000000,4588.000000
mean,2499.500000,231.101181,33.212947
std,1443.520003,235.665712,72.984099
min,0.000000,0.000000,0.000000
25%,1249.750000,0.000000,0.000000
50%,2499.500000,342.000000,0.000000
75%,3749.250000,425.000000,0.000000
max,4999.000000,1417.000000,275.000000


# class and kind matching
Now we want to see if the eu_class and the vehicle kind match. To do that, we drop the rows where columns and classes are null and hash the strings of kind and class.

In [4]:
x2= df['kind'].apply(lambda x: hash(x))
y2= df['eu_class'].apply(lambda x: hash(x))
correlation = x2.corr(y2)
print(correlation)

0.5540896909663889


Hmm.. still not good

In [5]:
df[["length", "width"]] = df[["length", "width"]].apply(pd.to_numeric)
print(df['length'].corr(df['width']))
correlation_df = df.corr() #calculate all correlations in df

print(correlation_df)

0.3253520978272152
            Unnamed: 0    length     width
Unnamed: 0    1.000000  0.008331  0.013174
length        0.008331  1.000000  0.325352
width         0.013174  0.325352  1.000000


In [6]:
df['kind'].unique()

array(['Personenauto', 'Motorfiets', 'Bromfiets', 'Bedrijfsauto',
       'Land- of bosbouwtrekker', 'Driewielig motorrijtuig',
       'Aanhangwagen', 'Middenasaanhangwagen',
       'Motorrijtuig met beperkte snelheid',
       'Land- of bosb aanhw of getr uitr stuk', 'Bus', 'Oplegger',
       'Motorfiets met zijspan', 'Autonome aanhangwagen',
       'Mobiele machine'], dtype=object)

In [7]:
len(df['eu_class'].unique())

25

In [8]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  5000 non-null   int64  
 1   plate       5000 non-null   object 
 2   kind        5000 non-null   object 
 3   make        5000 non-null   object 
 4   model       4937 non-null   object 
 5   length      4487 non-null   float64
 6   width       4588 non-null   float64
 7   eu_class    4998 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 312.6+ KB
None
